# WEkEO Harmonized Data Access (HDA) API - How-To

#### Load required libraries

In [1]:
import os
import sys
import json
import time
import base64
from IPython.core.display import HTML

import requests
import warnings
warnings.filterwarnings('ignore')

#### Load helper functions

In [2]:
from hda_api_functions import *

<hr>

### <a id='wekeo_dataset_id'></a>2. Get the Dataset ID 

In [3]:
dataset_id = "EO:MO:DAT:GLOBAL_ANALYSIS_FORECAST_PHY_001_024"

### <a id='wekeo_api_key'></a>3. Get the WEkEO API key

In order to interact with WEkEO's Harmonised Data Access API, each user requires an `API token`. This token can be generated from an `API Key`. Your current token can be found in your [WEkEO Dashboard](https://www.wekeo.eu/web/guest/dashboard) under Settings, but it is better to do it in a programmatic way, as shown below, as tokens have a limited lifetime of 1 hour!

The `API key` can generated by encoding your `username` and `password` to Base64. You can use the function [generate_api_key](./hda_api_functions.ipynb#generate_api_key) to programmatically generate your Base64-encoded api key. For this, you have to replace the 'username' and 'password' strings with your WEkEO username and password in the cell below.

Alternatively, you can go to this [website](https://www.base64encode.org/) that allows you to manually encode your `username:password` combination. An example of an encoded key is `wekeo-test:wekeo-test`, which is encoded to `d2VrZW8tdGVzdDp3ZWtlby10ZXN0`.

In [4]:
generate_api_key(user,password)

'xxxxxxxxxxxxxxxxx'

#### Alternative: enter manually the generated api key

In [5]:
api_key = 'xxxxxxxxxxxxxxxxx'

### <a id='wekeo_hda_request'></a>4. Initialise the Harmonised Data Access (HDA) API request

In order to initialise an API request, you have to initialise a dictionary that contains information on `dataset_id`, `api_key` and `download_directory_path`.

Please enter the path of the directory where the data shall be downloaded to.

In [6]:
# Enter here the directory path where you want to download the data to
download_dir_path = os.getcwd()

With `dataset_id`, `api_key` and `download_dir_path`, you can initialise the dictionary with the function [init](./hda_api_functions.ipynb#init).

In [7]:
hda_dict = init(dataset_id, api_key, download_dir_path)

#### Request access token

Once initialised, you can request an access token with the function [get_access_token](./hda_api_functions.ipynb#get_access_token). The access token is stored in the `hda_dict` dictionary.

You might need to accept the Terms and Conditions, which you can do with the function [acceptTandC](./hda_api_functions.ipynb#acceptTandC).

In [8]:
hda_dict = get_access_token(hda_dict)

Getting an access token. This token is valid for one hour only.
Success: Access token is 587f2be9-6895-3381-884d-a3009c808b25


#### Accept Terms and Conditions (if applicable)

In [9]:
hda_dict = acceptTandC(hda_dict)

Copernicus_General_License Terms and Conditions already accepted


### <a id='wekeo_json'></a>5. Load data descriptor file and request data

You can load the `JSON` file with `json.load()`. Alternatively, you can copy paste the dictionary describing your data into a cell, as done below. Make sure to choose one option if you are inputting your own example! 

In [42]:
try:
    with open('./forecastGS_2021.json', 'r') as f:
        data = json.load(f)
    print('Your JSON file:')
    print(data)
except:
    print('Your JSON file is not in the correct format, or is not found, please check it!')

Your JSON file:
{'datasetId': 'EO:MO:DAT:GLOBAL_ANALYSIS_FORECAST_PHY_001_024:global-analysis-forecast-phy-001-024-monthly', 'boundingBoxValues': [{'name': 'bbox', 'bbox': [-80.83495165780911, 24.76719560212698, -27.92561593949677, 53.18752483473007]}], 'dateRangeSelectValues': [{'name': 'position', 'start': '2021-10-01T00:00:00.000Z', 'end': '2022-01-01T00:00:00.000Z'}], 'multiStringSelectValues': [{'name': 'variable', 'value': ['uo', 'vo', 'thetao', 'so']}], 'stringChoiceValues': [{'name': 'service', 'value': 'GLOBAL_ANALYSIS_FORECAST_PHY_001_024-TDS'}, {'name': 'product', 'value': 'global-analysis-forecast-phy-001-024-monthly'}, {'name': 'startDepth', 'value': '0.49402499198913574'}, {'name': 'endDepth', 'value': '2225.077880859375'}]}


#### Initiate the request by assigning a job ID

The function [get_job_id](./hda_api_functions.ipynb#get_job_id) will launch your data request and your request is assigned a `job ID`.

In [43]:
hda_dict = get_job_id(hda_dict, data)

Query successfully submitted. Job ID is oOLWNmAEytsoKm-cFeYYzSZA5M4
Next check in 5 seconds
Query successfully submitted. Status is running
Next check in 10 seconds
Query successfully submitted. Status is completed


#### Build list of file names to be ordered and downloaded

The next step is to gather a list of file names available, based on your assigned `job ID`. The function [get_results_list](./hda_api_functions.ipynb#get_results_list) creates the list.

In [44]:
hda_dict = get_results_list(hda_dict)

&service=GLOBAL_ANALYSIS_FORECAST_PHY_001_024-TDS&product=global-analysis-forecast-phy-001-024-monthly&z_lo=0.49402499198913574&z_hi=2225.077880859375&t_lo=2021-10-01T00:00:00.000Z&t_hi=2022-01-01T00:00:00.000Z&x_lo=-80.83495165780911&y_hi=24.76719560212698&x_hi=-27.92561593949677&y_lo=53.18752483473007&variable=uo&variable=vo&variable=thetao&variable=so


#### Create an `order ID` for each file to be downloaded

The next step is to create an `order ID` for each file name to be downloaded. You can use the function [get_order_ids](./hda_api_functions.ipynb#get_order_ids).

In [45]:
hda_dict = get_order_ids(hda_dict)

Query successfully submitted. Order ID is 9CfILBDEbOBxhAPrDo6tlIjO9Kk
Next check in 5 seconds
Query successfully submitted. Status is completed


### <a id='wekeo_download'></a>6. Download requested data

As a final step, you can use the function [download_data](./hda_api_functions.ipynb#download_data) to initialize the data download and to download each file that has been assigned an `order ID`. 

In [46]:
hda_dict = download_data(hda_dict)

File size is:   204.12 MB
[==================================================]     45.45 Mbps[  204.12] MB downloaded, 46534.56 kbps
Download complete...
Time Elapsed: 4.491855793999999 seconds
Query successfully submitted. Response is <Response [200]>


<hr>